## Задача 2. Коллаборативная фильтрация (5 баллов)

### Вариант 1. По схожести пользователей

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
ratings_df = sqlContext.read.load(
    'file:///home/cloudera/Desktop/ml_data/ml_latest_small/ratings.csv',
    format='csv',
    header='true',
    inferSchema='true',
    sep=','
)
ratings_df.persist()
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



Разделите данные с рейтингами на обучающее (train - 0.8) и тестовое подмножества (test - 0.2).

In [7]:
df_train, df_test = ratings_df.randomSplit((0.8, 0.2), seed=123)
df_train.persist(), df_test.persist()
df_train.count(), df_test.count()

(80838, 19998)

Oпределите среднее значение рейтинга в обучающем подмножестве.

In [8]:
from pyspark.sql import functions as F

mean_movie_rating = df_train.select(F.mean('rating').alias('avg')).collect()[0]['avg']
mean_movie_rating

3.4985155496177542

Bычислите RMSE для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга.

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

df_test_with_prediction = df_test.withColumn('prediction', F.lit(mean_movie_rating))
df_test_with_prediction.show(5)

rmse_evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse_evaluator.evaluate(df_test_with_prediction)

+------+-------+------+---------+------------------+
|userId|movieId|rating|timestamp|        prediction|
+------+-------+------+---------+------------------+
|     1|      3|   4.0|964981247|3.4985155496177542|
|     1|      6|   4.0|964982224|3.4985155496177542|
|     1|     47|   5.0|964983815|3.4985155496177542|
|     1|    151|   5.0|964984041|3.4985155496177542|
|     1|    163|   5.0|964983650|3.4985155496177542|
+------+-------+------+---------+------------------+
only showing top 5 rows



1.0425819010275184

Реализуйте коллаборативную фильтрацию в соотвествии с вариантом.

Для определения схожести используйте train, для расчета RMSE - test.

In [15]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    coldStartStrategy='drop',
)
model = als.fit(df_train)

predictions = model.transform(df_test)
rmse = rmse_evaluator.evaluate(predictions)
rmse

1.0745717128399692

In [18]:
model.recommendForAllUsers(5).show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[799, 9.970222],...|
|   463|[[2937, 6.0137625...|
|   496|[[8638, 7.4388356...|
|   148|[[158872, 7.52323...|
|   540|[[158872, 7.45393...|
+------+--------------------+
only showing top 5 rows



## Задача 3. Факторизация матрицы (3 балла)
Выберите модель ALS по минимальному значению RMSE. Для этого используйте kfolds c k=4.

Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то RMSE будет NaN.

Поэтому заранее уберите из тестового/валидационного подмножества такие элементы.

In [21]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Параметры
ranks = (5, 10, 15)  # Kоличество факторов
regParams = (0.001, 0.01, 0.1, 1, 10)  # Pегуляризация
kfolds = 4

als = ALS(
    seed=123,
    maxIter=10,
    numUserBlocks=10,
    numItemBlocks=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    coldStartStrategy='drop',
)
paramsGrid = (
    ParamGridBuilder()
        .addGrid(als.rank, ranks)
        .addGrid(als.regParam, regParams)
        .build()
)
cross_validator = CrossValidator(estimator=als,
                                 estimatorParamMaps=paramsGrid,
                                 evaluator=rmse_evaluator,
                                 numFolds=kfolds)

cv_model = cross_validator.fit(df_train)
predictions = cv_model.transform(df_test)

rmse = rmse_evaluator.evaluate(predictions)
rmse

0.8838615498556199

Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов.

In [24]:
cv_model.bestModel.recommendForAllUsers(5).show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[33649, 4.576529...|
|   463|[[5466, 5.0053124...|
|   496|[[170705, 4.63225...|
|   148|[[33649, 4.688705...|
|   540|[[33649, 5.285536...|
+------+--------------------+
only showing top 5 rows

